## Benchmarking Economic Efficiency with Julia

Install requiered packages:

In [ ]:
using Pkg

Pkg.add("BenchmarkingEconomicEfficiency")
Pkg.add("DataFrames")
Pkg.add("CSV")

Or, instantiate environment to install required packages:

In [1]:
using Pkg
Pkg.instantiate()

Load required packages:

In [2]:
using BenchmarkingEconomicEfficiency
using DataFrames
using CSV

Increase the number of columens displayed in the notebook

In [3]:
ENV["COLUMNS"] = 100;

Load the data file:

In [4]:
df = DataFrame(CSV.File("DataBanks.csv"))
first(df, 5)

,Name,X1,X2,X3,W1,W2,W3,Y1,Y2,P1
,String31,Int64,Float64,Int64,Float64,Float64,Float64,Int64,Int64,Float64
1,Export-Import Bank,25019,201.999,505,0.0185859,1.71783,0.245545,3125,81996,0.00992
2,Bank of Taiwan,3171493,7950.96,76576,0.00769385,1.43404,0.0729602,904580,2091100,0.00930708
3,Taipei Fubon Bank,1222499,6434.01,12082,0.00500368,1.00357,0.590051,392491,866282,0.0132309
4,Bank of Kaohsiung,189169,913.995,2237,0.00545544,1.3895,0.249888,16740,163054,0.0242533
5,Land Bank,1846028,5732.01,22634,0.00663749,1.46563,0.193691,227086,1706964,0.0140872


In [5]:
describe(df)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,Name,,Bank SinoPac,,Union Bank,0,String31
2,X1,7.95536e5,25019,428995.0,3171493,0,Int64
3,X2,3826.19,201.999,3146.0,9537.98,0,Float64
4,X3,13393.2,505,8721.0,76576,0,Int64
5,W1,0.00639582,0.00249864,0.0061253,0.0185859,0,Float64
6,W2,1.25863,0.716977,1.21676,2.2963,0,Float64
7,W3,0.317114,0.0729602,0.301688,0.762522,0,Float64
8,Y1,1.96808e5,1681,157870.0,904580,0,Int64
9,Y2,6.09489e5,66947,328574.0,2091100,0,Int64


Extract names, inputs, output and prices as matrices:

In [6]:
X = [df.X1 df.X2 df.X3];
W = [df.W1 df.W2 df.W3];
Y = [df.Y1 df.Y2];
P = [df.P1 df.P2];
banks = df.Name;

## 3. Measuring and decomposing profit inefficiency

## 3.1. The Russell inefficiency measure

In [7]:
deaprofitrussell(X, Y, W, P, names = banks)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



Russell Profit DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
─────────────────────────────────────────────────────────────────
                                 Profit   Technical    Allocative
─────────────────────────────────────────────────────────────────
Export-Import Bank           3.03911e-5  4.81347e-6    2.55776e-5
Bank of Taiwan               0.179835    2.71764e-9    0.179835
Taipei Fubon Bank            0.220019    2.34592e-7    0.220019
Bank of Kaohsiung            4.95721     0.319545      4.63766
Land Bank                    1.3703e-7   5.04573e-8    8.65722e-8
Cooperative Bank             0.284738    2.97026e-8    0.284738
First Bank                   0.238689    1.19011e-8    0.238689
Hua Nan Bank                 0.240157    0.124564      0.115593
Chang Hwa Bank               0.44491     0.211199      0.233711
Mega Bank                    9.61495e-8  2.11586e-8    7.49909e-8
Cathay United Bank           0.814791    0.334818   

In [8]:
dearussell(X, Y, orient = :Graph, rts  = :VRS, names = banks)

Russell DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────
                           efficiency     effX1     effX2      effX3    effY1    effY2
──────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           0.999995  1.0       1.0       0.999997   1.00002  1.0
Bank of Taiwan               1.0       1.0       1.0       1.0        1.0      1.0
Taipei Fubon Bank            1.0       1.0       0.999999  1.0        1.0      1.0
Bank of Kaohsiung            0.680455  0.714658  0.623646  0.653934   2.43882  1.0
Land Bank                    1.0       1.0       1.0       1.0        1.0      1.0
Cooperative Bank             1.0       1.0       1.0       1.0        1.0      1.0
First Bank                   1.0       1.0       1.0       1.0        1.0      1.0
Hua Nan Bank                 0.875436  0.960964  0.776304  0

## 3.2. The Weighted Additive Distance Function

In [9]:
deaprofitadd(X, Y, W, P, :MIP, names = banks)

Profit Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Weights = MIP; Returns to Scale = VRS
─────────────────────────────────────────────────────────────────────
                                  Profit     Technical     Allocative
─────────────────────────────────────────────────────────────────────
Export-Import Bank           0.0          -1.78195e-16    1.78195e-16
Bank of Taiwan               0.899175     -2.52669e-16    0.899175
Taipei Fubon Bank            1.1001        0.0            1.1001
Bank of Kaohsiung           24.786         7.07646       17.7096
Land Bank                    1.59211e-14   0.0            1.59211e-14
Cooperative Bank             1.42369       0.0            1.42369
First Bank                   1.19344       7.69353e-18    1.19344
Hua Nan Bank                 1.20078       0.622821       0.577963
Chang Hwa Bank               2.22455       1.10356        1.12099
Mega Bank                   -2.15648e-15   0.0           -2.15648e-15
Cathay United Bank

In [10]:
deaadd(X, Y, :MIP, names = banks)

Weighted Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
Weights = MIP
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency          slackX1    slackX2          slackX3         slackY1        slackY2
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank         -1.78195e-16      0.0             0.0        -8.99886e-14       0.0            0.0
Bank of Taiwan             -2.52669e-16     -1.98653e-10     0.0        -1.45519e-11       0.0            0.0
Taipei Fubon Bank           0.0              0.0             0.0         0.0               0.0            0.0
Bank of Kaohsiung           7.07646       1680.17            0.0       129.166             1.17345e5      0.0
Land Bank                   0.0              0.0             0.0         0.0              

## 3.3 The Enhanced Russell Graph (or Slack-Based Measure) inefficiency measure

In [11]:
deaprofiterg(X, Y, W, P, names = banks)

Enhanced Russell Graph Slack Based Measure Profit DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
───────────────────────────────────────────────────────────────────
                                 Profit     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank          0.0           0.0           0.0
Bank of Taiwan              0.299725      1.11022e-16   0.299725
Taipei Fubon Bank           0.550048      2.22045e-16   0.550048
Bank of Kaohsiung           2.75099       0.78295       1.96804
Land Bank                   7.96057e-15  -2.22045e-16   8.18261e-15
Cooperative Bank            0.711845      0.0           0.711845
First Bank                  0.564601      0.0           0.564601
Hua Nan Bank                0.558457      0.207607      0.35085
Chang Hwa Bank              0.986781      0.36396       0.622822
Mega Bank                  -1.07824e-15   0.0          -1.07824e-15
Cathay United Bank          

In [12]:
deaerg(X, Y, rts  = :VRS, names = banks)

Enhanced Russell Graph Slack Based Measure DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency       beta          slackX1         slackX2          slackX3         slackY1         slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          1.0        1.0            0.0             0.0              0.0               0.0           0.0
Bank of Taiwan              1.0        1.0            2.07846e-10     0.0              1.80339e-11       0.0           0.0
Taipei Fubon Bank           1.0        1.0            2.1264e-10      3.08885e-12      0.0               0.0           0.0
Bank of Kaohsiung           0.21705    0.221979    1680.17            0.0            129.166             

## 3.4. The Directional Distance Function

In [13]:
deaprofit(X, Y, W, P, Gx = :Mean, Gy = :Mean,  names = banks)

Profit DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Mean; Gy = Mean
───────────────────────────────────────────────────────────────────
                                 Profit     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank          0.0           0.0           0.0
Bank of Taiwan              0.210065     -7.08067e-17   0.210065
Taipei Fubon Bank           0.201694      0.0           0.201694
Bank of Kaohsiung           0.36845       0.0418304     0.32662
Land Bank                   1.93201e-15   0.0           1.93201e-15
Cooperative Bank            0.207178      0.0           0.207178
First Bank                  0.231552      0.0           0.231552
Hua Nan Bank                0.244772      0.0561266     0.188645
Chang Hwa Bank              0.349622      0.051373      0.298249
Mega Bank                  -3.10894e-16   0.0          -3.10894e-16
Cathay United Bank          0.862221      0.25469

In [14]:
deaddf(X, Y, rts  = :VRS, Gx = :Mean, Gy = :Mean, names = banks)

Directional DF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Mean; Gy = Mean
────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency          slackX1   slackX2         slackX3          slackY1  slackY2
────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          0.0              0.0             0.0       0.0             -9.10433e-12      0.0
Bank of Taiwan             -7.08067e-17     -1.52296e-10     0.0      -1.45519e-11      0.0              0.0
Taipei Fubon Bank           0.0              0.0             0.0       0.0              0.0              0.0
Bank of Kaohsiung           0.0418304        0.0           148.877     0.0           4758.7              0.0
Land Bank                   0.0              0.0             0.0       0.0              0.0              0.0
Cooperative Bank       

## 3.5. The Hölder Distance Function}

In [15]:
deaprofitholder(X, Y, W, P, l = Inf, weight = true, names = banks)

Profit Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
───────────────────────────────────────────────────────────────────
                                 Profit     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank          0.0          -9.77224e-17   9.77224e-17
Bank of Taiwan              0.0609693    -2.04769e-17   0.0609693
Taipei Fubon Bank           0.146089      0.0           0.146089
Bank of Kaohsiung           1.72965       0.178596      1.55106
Land Bank                   9.00745e-16   0.0           9.00745e-16
Cooperative Bank            0.0762826     0.0           0.0762826
First Bank                  0.115133      0.0           0.115133
Hua Nan Bank                0.123537      0.0276888     0.0958478
Chang Hwa Bank              0.211342      0.0316473     0.179695
Mega Bank                  -1.87802e-16   0.0          -1.87802e-16
Cat

In [16]:
deaholder(X, Y, l = Inf, weight = true, orient = :Graph, rts = :VRS, names = banks)

Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency          slackX1   slackX2         slackX3          slackY1      slackY2
────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank         -9.77224e-17      0.0             0.0       0.0              1.53736e-11  0.0
Bank of Taiwan             -2.04769e-17     -1.52296e-10     0.0      -1.45519e-11      0.0          0.0
Taipei Fubon Bank           0.0              0.0             0.0       0.0              0.0          0.0
Bank of Kaohsiung           0.178596         0.0           149.48      0.0           5748.91         0.0
Land Bank                   0.0              0.0             0.0       0.0              0.0    

## 3.6. The Modified Distance Function

In [17]:
deaprofitmddf(X, Y, W, P, Gx = :Observed, Gy =  :Observed,  names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: INFEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deamddf.jl:177


Profit Modified DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Observed; Gy = Observed
─────────────────────────────────────────────────────────────────────
                                  Profit     Technical     Allocative
─────────────────────────────────────────────────────────────────────
Export-Import Bank           0.0           0.0            0.0
Bank of Taiwan               0.596709      3.42411e-18    0.596709
Taipei Fubon Bank            1.1001        0.0            1.1001
Bank of Kaohsiung            9.75109       0.361837       9.38925
Land Bank                    4.15667e-15   0.0            4.15667e-15
Cooperative Bank             0.343185      0.0            0.343185
First Bank                   0.716428     -7.11564e-18    0.716428
Hua Nan Bank                 0.71398       0.0563877      0.657592
Chang Hwa Bank               1.33121       0.0649374      1.26627
Mega Bank                   -1.12099e-15   0.0           -1.12099e-15
Catha

In [18]:
deamddf(X, Y, rts = :VRS, Gx = :Observed, Gy= :Observed, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: FEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deamddf.jl:177


Modified DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Observed; Gy = Observed
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency            βx        βy          slackX1   slackX2         slackX3          slackY1  slackY2
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          0.0           0.0          0.0           0.0             0.0       0.0             -9.10433e-12      0.0
Bank of Taiwan              3.42411e-18   3.42411e-18  0.0          -1.52296e-10     0.0      -1.45519e-11      0.0              0.0
Taipei Fubon Bank           0.0           0.0          0.0           0.0             0.0       0.0              0.0              0.0
Bank of Kaohsiung           0.361837      0.0          0.361837      0.0           209.97  

## 3.7. The Reverse Directional Distance Function

In [19]:
deaprofitrddf(X, Y, W, P, :ERG,  names = banks)

Profit Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Associated efficiency measure = ERG
───────────────────────────────────────────────────────────────────
                                 Profit     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank          0.0           0.0           0.0
Bank of Taiwan              0.299725     -4.82974e-13   0.299725
Taipei Fubon Bank           0.550048      0.0           0.550048
Bank of Kaohsiung           2.72869       0.78295       1.94574
Land Bank                   7.96057e-15   0.0           7.96057e-15
Cooperative Bank            0.711845      0.0           0.711845
First Bank                  0.564601     -1.98624e-13   0.564601
Hua Nan Bank                0.293138      0.207607      0.085531
Chang Hwa Bank              0.535171      0.36396       0.171212
Mega Bank                  -1.07824e-15   0.0          -1.07824e-15
Cathay United Bank    

In [20]:
dearddf(X, Y, :ERG, rts  = :VRS, names = banks)

Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
Associated efficiency measure = ERG
───────────────────────────────────────
                             efficiency
───────────────────────────────────────
Export-Import Bank          0.0
Bank of Taiwan             -4.82974e-13
Taipei Fubon Bank           0.0
Bank of Kaohsiung           0.78295
Land Bank                   0.0
Cooperative Bank            0.0
First Bank                 -1.98624e-13
Hua Nan Bank                0.207607
Chang Hwa Bank              0.36396
Mega Bank                   0.0
Cathay United Bank          0.593035
The Shanghai Bank           0.257427
Union Bank                  0.933576
Far Eastern Bank            0.0
E. Sun Bank                 0.217258
Cosmos Bank                 0.988055
Taishin Bank                0.532162
Ta Chong Bank               0.41594
Jih Sun Bank                0.812509
Entie Bank                  0.735329
China Trust Bank            0

## 3.8. The General Direct Approach

In [21]:
deaprofitgda(X, Y, W, P, :ERG, names = banks)

General Direct Approach Profit DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Associated efficiency measure = ERG
───────────────────────────────────────────────────────────────────
                                 Profit     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank          0.0           0.0           0.0
Bank of Taiwan              0.299725      1.11022e-16   0.299725
Taipei Fubon Bank           0.550048      2.22045e-16   0.550048
Bank of Kaohsiung           2.72869       0.78295       1.94574
Land Bank                   7.96057e-15  -2.22045e-16   7.96057e-15
Cooperative Bank            0.711845      0.0           0.711845
First Bank                  0.564601      0.0           0.564601
Hua Nan Bank                0.293138      0.207607      0.085531
Chang Hwa Bank              0.535171      0.36396       0.171212
Mega Bank                  -1.07824e-15   0.0          -1.07824e-15
Cathay Uni

In [22]:
deaerg(X, Y, rts = :VRS,  names = banks)

Enhanced Russell Graph Slack Based Measure DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Graph; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency       beta          slackX1         slackX2          slackX3         slackY1         slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          1.0        1.0            0.0             0.0              0.0               0.0           0.0
Bank of Taiwan              1.0        1.0            2.07846e-10     0.0              1.80339e-11       0.0           0.0
Taipei Fubon Bank           1.0        1.0            2.1264e-10      3.08885e-12      0.0               0.0           0.0
Bank of Kaohsiung           0.21705    0.221979    1680.17            0.0            129.166             

# 4. Measuring and decomposing profitability efficiency

In [23]:
deaprofitability(X, Y, W, P, alpha = 0.5, names = banks)

Profitability DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
alpha = 0.5; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────
                           Profitability       CRS       VRS     Scale  Allocative
──────────────────────────────────────────────────────────────────────────────────
Export-Import Bank             0.999999   0.999997  0.999999  0.999998    1.0
Bank of Taiwan                 0.393451   1.0       1.0       1.0         0.393451
Taipei Fubon Bank              0.444591   0.614278  1.0       0.614278    0.723762
Bank of Kaohsiung              0.400458   0.48107   0.700667  0.686588    0.832432
Land Bank                      0.501915   0.830479  1.0       0.830479    0.604368
Cooperative Bank               0.392848   0.671643  1.0       0.671643    0.584905
First Bank                     0.417051   0.666708  1.0       0.666708    0.625538
Hua Nan Bank                   0.411813   0.602187  0.946148  0.636462    0.683

In [24]:
deagdf(X, Y, alpha = 0.5, rts = :CRS, names = banks)

┌ Warning: DMU 28 termination status: ALMOST_LOCALLY_SOLVED. Primal status: NEARLY_FEASIBLE_POINT. Dual status: NEARLY_FEASIBLE_POINT
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaadd.jl:217


Generalized DF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
alpha = 0.5; Returns to Scale = CRS
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency          slackX1         slackX2        slackX3        slackY1      slackY2
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           0.999997       0.0963408     -9.997e-9        0.00128176    -8.3712e-9    0.00622012
Bank of Taiwan               1.0           21.5583        -9.99997e-9      0.585936      -9.99132e-9  -9.98696e-9
Taipei Fubon Bank            0.614278       2.75815e5      0.00250125     -9.99319e-9    -8.8296e-9   -6.82675e-9
Bank of Kaohsiung            0.48107    51656.5            0.000468985    -9.9893e-9     -7.79733e-9  -6.342e-9
Land Bank                    0.830479       7.34806e5     -9.99997e-9      0.411265      -9.99132e-9  -9.98

In [25]:
deagdf(X, Y, alpha = 0.5, rts = :VRS, names = banks)

Generalized DF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
alpha = 0.5; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency         slackX1         slackX2        slackX3         slackY1       slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           0.999999     -7.06512e-9     0.000166836     0.00128651      0.0080689   -7.38061e-9
Bank of Taiwan               1.0           0.130379      -9.96657e-9      0.00862377     -6.06987e-9  -8.3297e-11
Taipei Fubon Bank            1.0           0.242837       0.00402907     -9.99152e-9     -9.85056e-9  -8.93276e-9
Bank of Kaohsiung            0.700667     -6.90948e-9   154.782          -9.91863e-9   5868.52        -8.11283e-9
Land Bank                    1.0           0.022368      -9.8476e-9       0.00134814      0.053665    

# 5. Measuring and decomposing cost and revenue efficiency

## 5.1. Multiplicative models based on the radial input-oriented and output-oriented measures

### Cost efficiency

In [28]:
deacost(X, Y, W, names = banks)

Cost DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
──────────────────────────────────────────────────────────
                               Cost  Technical  Allocative
──────────────────────────────────────────────────────────
Export-Import Bank         1.0        1.0         1.0
Bank of Taiwan             1.0        1.0         1.0
Taipei Fubon Bank          0.9216     1.0         0.9216
Bank of Kaohsiung          0.597026   0.647333    0.922285
Land Bank                  1.0        1.0         1.0
Cooperative Bank           0.960035   1.0         0.960035
First Bank                 0.989871   1.0         0.989871
Hua Nan Bank               0.81141    0.943612    0.859898
Chang Hwa Bank             0.70393    0.935063    0.752816
Mega Bank                  1.0        1.0         1.0
Cathay United Bank         0.543989   0.709586    0.766629
The Shanghai Bank          0.787542   0.886617    0.888255
Union Bank                 0.31538    0.49

In [29]:
dea(X, Y, orient = :Input, rts = :VRS, names = banks)

Radial DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency          slackX1    slackX2         slackX3          slackY1          slackY2
───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           1.0           0.0             0.0        0.0             -2.19254e-11      0.0
Bank of Taiwan               1.0          -1.52296e-10     0.0       -1.45519e-11      0.0              0.0
Taipei Fubon Bank            1.0           0.0             0.0        0.0              0.0              0.0
Bank of Kaohsiung            0.647333      0.0            90.5226     0.0           4418.74             0.0
Land Bank                    1.0           0.0             0.0        0.0              0.0              0.0
Cooperative Ban

### Revenue efficiency

In [30]:
dearevenue(X, Y, P, names = banks)

Revenue DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
──────────────────────────────────────────────────────────
                            Revenue  Technical  Allocative
──────────────────────────────────────────────────────────
Export-Import Bank         1.0        1.0         1.0
Bank of Taiwan             1.0        1.0         1.0
Taipei Fubon Bank          0.919161   1.0         0.919161
Bank of Kaohsiung          0.507508   0.734302    0.691143
Land Bank                  1.0        1.0         1.0
Cooperative Bank           1.0        1.0         1.0
First Bank                 0.964614   1.0         0.964614
Hua Nan Bank               0.944698   0.948404    0.996092
Chang Hwa Bank             0.927525   0.941856    0.984784
Mega Bank                  1.0        1.0         1.0
Cathay United Bank         0.571303   0.722436    0.7908
The Shanghai Bank          0.836444   0.901161    0.928185
Union Bank                 0.10421    0.591

In [31]:
dea(X, Y, orient = :Output, rts = :VRS, names = banks)

Radial DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency          slackX1         slackX2         slackX3          slackY1  slackY2
────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank            1.0          0.0             0.0             0.0             -9.10433e-12      0.0
Bank of Taiwan                1.0         -1.0835e-9       0.0            -4.36557e-11      0.0              0.0
Taipei Fubon Bank             1.0          1.28176e-9      1.52105e-11     0.0              0.0              0.0
Bank of Kaohsiung             1.36184      0.0           209.97            0.0           7113.67             0.0
Land Bank                     1.0          0.0             0.0             0.0              0.0              0.

## 5.2. Additive models based on inefficiency measures

### The Russell inefficiency measure

In [32]:
deacostrussell(X, Y, W, names = banks)

Russell Cost DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
────────────────────────────────────────────────────────────────
                                   Cost  Technical    Allocative
────────────────────────────────────────────────────────────────
Export-Import Bank         -9.16829e-16   0.0       -9.16829e-16
Bank of Taiwan             -1.56276e-14   0.0       -1.56276e-14
Taipei Fubon Bank           0.084176      0.0        0.084176
Bank of Kaohsiung           0.687483      0.387981   0.299501
Land Bank                   5.53221e-16   0.0        5.53221e-16
Cooperative Bank            0.0792926     0.0        0.0792926
First Bank                  0.0157307     0.0        0.0157307
Hua Nan Bank                0.287933      0.207607   0.0803257
Chang Hwa Bank              0.464024      0.31642    0.147603
Mega Bank                  -5.46585e-15   0.0       -5.46585e-15
Cathay United Bank          0.490611      0.44973    0.0408809
The 

In [33]:
dearevenuerussell(X, Y, P, names = banks)

Russell Revenue DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
─────────────────────────────────────────────────────────────
                              Revenue  Technical   Allocative
─────────────────────────────────────────────────────────────
Export-Import Bank          0.0         0.0       0.0
Bank of Taiwan              0.0         0.0       0.0
Taipei Fubon Bank           0.164297    0.0       0.164297
Bank of Kaohsiung           3.53388     3.53223   0.0016523
Land Bank                   0.0         0.0       0.0
Cooperative Bank            0.0         0.0       0.0
First Bank                  0.0990394   0.0       0.0990394
Hua Nan Bank                0.139932    0.139545  0.000386895
Chang Hwa Bank              0.219293    0.195248  0.0240449
Mega Bank                   0.0         0.0       0.0
Cathay United Bank          1.10549     1.05484   0.05065
The Shanghai Bank           0.420213    0.124158  0.296056
Union Bank         

In [34]:
dearussell(X, Y, orient = :Input, rts  = :VRS, names = banks)

Russell DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency     effX1     effX2      effX3          slackY1  slackY2
──────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           1.0       1.0       1.0       1.0           -9.10433e-12      0.0
Bank of Taiwan               1.0       1.0       1.0       1.0            0.0              0.0
Taipei Fubon Bank            1.0       1.0       1.0       1.0            0.0              0.0
Bank of Kaohsiung            0.612019  0.668261  0.563576  0.604219    8535.19             0.0
Land Bank                    1.0       1.0       1.0       1.0            0.0              0.0
Cooperative Bank             1.0       1.0       1.0       1.0            0.0              0.0
First Bank                   1.0       1.0     

In [35]:
dearussell(X, Y, orient = :Output, rts  = :VRS, names = banks)

Russell DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
─────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency     effY1    effY2          slackX1     slackX2         slackX3
─────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank            1.0       1.0      1.0          0.0             0.0         0.0
Bank of Taiwan                1.0       1.0      1.0         -1.52296e-10     0.0        -1.45519e-11
Taipei Fubon Bank             1.0       1.0      1.0          0.0             0.0         0.0
Bank of Kaohsiung             4.53223   8.06446  1.0        147.515           0.0        -1.47978e-12
Land Bank                     1.0       1.0      1.0          0.0             0.0         0.0
Cooperative Bank              1.0       1.0      1.0          0.0             0.0         0.0
First Bank     

### The Weighted Additive Distance Function

In [36]:
deacostadd(X, Y, W, :MIP, names = banks)

Cost Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Weights = MIP
───────────────────────────────────────────────────────────────────
                                   Cost     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank         -2.75049e-15  -1.78195e-16  -2.57229e-15
Bank of Taiwan             -4.68828e-14  -2.52669e-16  -4.66302e-14
Taipei Fubon Bank           0.252528      0.0           0.252528
Bank of Kaohsiung           2.06245       1.16394       0.898504
Land Bank                   1.65966e-15   0.0           1.65966e-15
Cooperative Bank            0.237878      0.0           0.237878
First Bank                  0.047192      7.69353e-18   0.047192
Hua Nan Bank                0.863798      0.622821      0.240977
Chang Hwa Bank              1.39207       0.94926       0.44281
Mega Bank                  -1.63976e-14   0.0          -1.63976e-14
Cathay United Ban

In [37]:
dearevenueadd(X, Y, P, :MIP, names = banks)

Revenue Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Weights = MIP
─────────────────────────────────────────────────────────────────
                              Revenue     Technical    Allocative
─────────────────────────────────────────────────────────────────
Export-Import Bank          0.0        -2.91339e-15   2.91339e-15
Bank of Taiwan              0.0         1.9367e-18   -1.9367e-18
Taipei Fubon Bank           0.328594    0.0           0.328594
Bank of Kaohsiung           7.06777     7.06446       0.00330459
Land Bank                   0.0         0.0           0.0
Cooperative Bank            0.0         0.0           0.0
First Bank                  0.198079   -1.94126e-17   0.198079
Hua Nan Bank                0.279865    0.279091      0.00077379
Chang Hwa Bank              0.438585    0.390496      0.0480898
Mega Bank                   0.0         0.0           0.0
Cathay United Bank          2.21098     2.10968       0

In [38]:
deaadd(X, Y, orient = :Input, rts  = :VRS, names = banks)

Weighted Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Weights = Ones
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 efficiency           slackX1         slackX2          slackX3    slackY1  slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank             -7.00546e-12      -6.93771e-12    -5.22354e-16     -6.72259e-14      0.0        0.0
Bank of Taiwan                 -1.66848e-10      -1.52296e-10     0.0             -1.45519e-11      0.0        0.0
Taipei Fubon Bank               0.0               0.0             0.0              0.0              0.0        0.0
Bank of Kaohsiung           73051.5           72004.1           421.091          626.316            0.0        0.0
Land Bank                       0.0               0.0             0.0   

In [39]:
deaadd(X, Y, orient = :Output, rts  = :VRS, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: INFEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaadd.jl:217


Weighted Additive DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Weights = Ones
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                 efficiency   slackX1     slackX2         slackX3           slackY1           slackY2
─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank             -9.10433e-12      0.0      0.0         0.0              -9.10433e-12       0.0
Bank of Taiwan                 -1.22974e-10      0.0      0.0        -8.96205e-12       0.0              -1.22974e-10
Taipei Fubon Bank               0.0              0.0      0.0         0.0               0.0               0.0
Bank of Kaohsiung               1.18743e5        0.0      0.0         0.0               1.14518e5      4224.91
Land Bank                       0.0              0.0      0.0         0.0

### The Directional Distance Function

In [40]:
deacostddf(X, Y, W, Gx = :Ones, names = banks)

Cost DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Gx = Ones
─────────────────────────────────────────────────────────────────────────
                                     Cost       Technical      Allocative
─────────────────────────────────────────────────────────────────────────
Export-Import Bank           -1.72083e-13     0.0            -1.72083e-13
Bank of Taiwan               -1.72929e-10    -3.29584e-13    -1.72599e-10
Taipei Fubon Bank           966.275           0.0           966.275
Bank of Kaohsiung           700.921         432.577         268.344
Land Bank                     4.36743e-12     0.0             4.36743e-12
Cooperative Bank            833.586           0.0           833.586
First Bank                  144.041          -2.06159e-13   144.041
Hua Nan Bank               2808.01         2297.68          510.328
Chang Hwa Bank             3758.22         2660.23         1097.98
Mega Bank                    -4.74366e-1

In [41]:
dearevenueddf(X, Y, P, Gy = :Ones, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: FEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaddf.jl:173


Revenue DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Gy = Ones
───────────────────────────────────────────────────────────────────────
                                 Revenue      Technical      Allocative
───────────────────────────────────────────────────────────────────────
Export-Import Bank             0.0            0.0             0.0
Bank of Taiwan                 0.0            0.0             0.0
Taipei Fubon Bank          57583.8            0.0         57583.8
Bank of Kaohsiung          71920.5        45282.4         26638.1
Land Bank                      0.0            0.0             0.0
Cooperative Bank               0.0            0.0             0.0
First Bank                 36268.7            0.0         36268.7
Hua Nan Bank               50398.0        47862.9          2535.04
Chang Hwa Bank             53218.0        43809.2          9408.75
Mega Bank                      0.0            0.0             0.0
Cathay Un

In [42]:
deaddf(X, Y, rts = :VRS, Gx = :Ones, Gy = :Zeros, names = banks)

Directional DF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Ones; Gy = Zeros
──────────────────────────────────────────────────────────────────────────────────────────────────────────────
                               efficiency          slackX1   slackX2         slackX3          slackY1  slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank            0.0              0.0             0.0       0.0             -9.10433e-12      0.0
Bank of Taiwan               -3.29584e-13     -1.52296e-10     0.0      -1.45519e-11      0.0              0.0
Taipei Fubon Bank             0.0              0.0             0.0       0.0              0.0              0.0
Bank of Kaohsiung           432.577        69379.2             0.0       5.01316          0.0              0.0
Land Bank                     0.0              0.0             0.0       0.0              0.0              0.0
Cooper

In [43]:
deaddf(X, Y, rts = :VRS, Gx = :Zeros, Gy = :Ones, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: INFEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaddf.jl:173


Directional DF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Gx = Zeros; Gy = Ones
───────────────────────────────────────────────────────────────────────────────────────────────────────────
                              efficiency          slackX1    slackX2         slackX3       slackY1  slackY2
───────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank             0.0            0.0             0.0        0.0          -9.10433e-12      0.0
Bank of Taiwan                 0.0           -1.52296e-10     0.0       -1.45519e-11   0.0              0.0
Taipei Fubon Bank              0.0            0.0             0.0        0.0           0.0              0.0
Bank of Kaohsiung          45282.4            0.0            28.6909     0.0          -5.32964e-10      0.0
Land Bank                      0.0            0.0             0.0        0.0           0.0              0.0
Cooperative Bank              

### The Hölder Distance Function

In [44]:
deacostholder(X, Y, W, l = Inf, weight = true, names = banks)

Cost Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
───────────────────────────────────────────────────────────────────
                                   Cost     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank         -3.64381e-16   0.0          -3.64381e-16
Bank of Taiwan             -6.32845e-15  -4.15853e-17  -6.28686e-15
Taipei Fubon Bank           0.0783999     0.0           0.0783999
Bank of Kaohsiung           0.402974      0.352667      0.0503073
Land Bank                   2.90597e-16   0.0           2.90597e-16
Cooperative Bank            0.0399645     0.0           0.0399645
First Bank                  0.010129      0.0           0.010129
Hua Nan Bank                0.18859       0.0563877     0.132202
Chang Hwa Bank              0.29607       0.0649374     0.231132
Mega Bank                  -4.57527e-15   0.0   

In [45]:
dearevenueholder(X, Y, P, l = Inf, weight = true, names = banks)

Revenue Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
────────────────────────────────────────────────────────────────
                              Revenue     Technical   Allocative
────────────────────────────────────────────────────────────────
Export-Import Bank         0.0          0.0          0.0
Bank of Taiwan             0.0          0.0          0.0
Taipei Fubon Bank          0.0879491    1.34385e-16  0.0879491
Bank of Kaohsiung          0.970414     0.361837     0.608577
Land Bank                  0.0          0.0          0.0
Cooperative Bank           0.0          0.0          0.0
First Bank                 0.0366845    0.0          0.0366845
Hua Nan Bank               0.0585395    0.0544031    0.00413647
Chang Hwa Bank             0.0781379    0.061733     0.0164048
Mega Bank                  0.0         -5.63202e-17  5.63202e-17
Cathay United Bank         0.750385     0.

In [46]:
deaholder(X, Y, l = Inf, weight = true, orient = :Input, rts = :VRS, names = banks)

Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
─────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency          slackX1    slackX2         slackX3          slackY1  slackY2
─────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          0.0              0.0             0.0        0.0             -9.10433e-12      0.0
Bank of Taiwan             -4.15853e-17     -1.52296e-10     0.0       -1.45519e-11      0.0              0.0
Taipei Fubon Bank           0.0              0.0             0.0        0.0              0.0              0.0
Bank of Kaohsiung           0.352667         0.0            90.5226     0.0           4418.74             0.0
Land Bank                   0.0              0.0             0.0        0.0         

In [47]:
deaholder(X, Y, l = Inf, weight = true, orient = :Output, rts = :VRS, names = banks)

Hölder LInf DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Weighted (weakly) Hölder distance function 
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                             efficiency          slackX1         slackX2         slackX3          slackY1  slackY2
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank          0.0              0.0             0.0             0.0             -9.10433e-12      0.0
Bank of Taiwan              0.0             -1.52296e-10     0.0            -1.45519e-11      0.0              0.0
Taipei Fubon Bank           1.34385e-16     -5.54318e-10    -7.41219e-12     0.0              0.0              0.0
Bank of Kaohsiung           0.361837         0.0           209.97            0.0           7113.67             0.0
Land Bank                   0.0              0.0

### The Reverse Directional Distance Function

In [48]:
deacostrddf(X, Y, W, :ERG, rts = :VRS, names = banks)

Cost Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale: VRS
Associated efficiency measure = ERG
───────────────────────────────────────────────────────────────────
                                   Cost     Technical    Allocative
───────────────────────────────────────────────────────────────────
Export-Import Bank         -9.16829e-16   0.0          -9.16829e-16
Bank of Taiwan             -1.56276e-14  -3.29584e-13   3.13956e-13
Taipei Fubon Bank           0.084176      0.0           0.084176
Bank of Kaohsiung           0.400148      0.387981      0.0121665
Land Bank                   5.53221e-16   0.0           5.53221e-16
Cooperative Bank            0.0792926     0.0           0.0792926
First Bank                  0.0157307    -2.06159e-13   0.0157307
Hua Nan Bank                0.210872      0.207607      0.00326531
Chang Hwa Bank              0.31642       0.31642      -2.22045e-16
Mega Bank                  -5.46585e-15   0.0          -5.46585e-15
Catha

In [49]:
dearevenuerddf(X, Y, P, :ERG, rts = :VRS, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: INFEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaddf.jl:173


Revenue Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale: VRS
Associated efficiency measure = ERG
─────────────────────────────────────────────────────────────
                             Revenue  Technical    Allocative
─────────────────────────────────────────────────────────────
Export-Import Bank         0.0         0.0        0.0
Bank of Taiwan             0.0         0.0        0.0
Taipei Fubon Bank          0.164297    0.0        0.164297
Bank of Kaohsiung          0.779723    0.779358   0.000364566
Land Bank                  0.0         0.0        0.0
Cooperative Bank           0.0         0.0        0.0
First Bank                 0.0990394   0.0        0.0990394
Hua Nan Bank               0.122797    0.122457   0.000339517
Chang Hwa Bank             0.18347     0.163353   0.0201171
Mega Bank                  0.0         0.0        0.0
Cathay United Bank         0.515661    0.513344   0.00231686
The Shanghai Bank          0.265163    0.110445   0.1547

In [50]:
dearddf(X, Y, :ERG, orient = :Input, rts = :VRS, names = banks)

Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
Associated efficiency measure = ERG
───────────────────────────────────────
                             efficiency
───────────────────────────────────────
Export-Import Bank          0.0
Bank of Taiwan             -3.29584e-13
Taipei Fubon Bank           0.0
Bank of Kaohsiung           0.387981
Land Bank                   0.0
Cooperative Bank            0.0
First Bank                 -2.06159e-13
Hua Nan Bank                0.207607
Chang Hwa Bank              0.31642
Mega Bank                   0.0
Cathay United Bank          0.44973
The Shanghai Bank           0.256019
Union Bank                  0.686893
Far Eastern Bank            0.0
E. Sun Bank                 0.217258
Cosmos Bank                 0.856719
Taishin Bank                0.480661
Ta Chong Bank               0.313858
Jih Sun Bank                0.65638
Entie Bank                  0.428805
China Trust Bank            0

In [51]:
dearddf(X, Y, :ERG, orient = :Output, rts = :VRS, names = banks)

┌ Warning: DMU 29 termination status: TIME_LIMIT. Primal status: INFEASIBLE_POINT. Dual status: NO_SOLUTION
└ @ DataEnvelopmentAnalysis /Users/javierbarbero/.julia/packages/DataEnvelopmentAnalysis/CcljZ/src/deaddf.jl:173


Reverse DDF DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
Associated efficiency measure = ERG
──────────────────────────────────────
                            efficiency
──────────────────────────────────────
Export-Import Bank            0.0
Bank of Taiwan                0.0
Taipei Fubon Bank             0.0
Bank of Kaohsiung             0.779358
Land Bank                     0.0
Cooperative Bank              0.0
First Bank                    0.0
Hua Nan Bank                  0.122457
Chang Hwa Bank                0.163353
Mega Bank                     0.0
Cathay United Bank            0.513344
The Shanghai Bank             0.110445
Union Bank                    0.898641
Far Eastern Bank              0.0
E. Sun Bank                   0.127932
Cosmos Bank                   0.975656
Taishin Bank                  0.291503
Ta Chong Bank                 0.303661
Jih Sun Bank                  0.748352
Entie Bank                    0.694196
Chin

### The General Direct Approach

In [52]:
deacostgda(X, Y, W, :ERG, names = banks)

General Direct Approach Cost DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Associated efficiency measure = ERG
────────────────────────────────────────────────────────────────
                                   Cost  Technical    Allocative
────────────────────────────────────────────────────────────────
Export-Import Bank         -9.16829e-16   0.0        3.0561e-16
Bank of Taiwan             -1.56276e-14   0.0       -8.24791e-15
Taipei Fubon Bank           0.084176      0.0        0.084176
Bank of Kaohsiung           0.400148      0.387981   0.0121665
Land Bank                   5.53221e-16   0.0       -1.93627e-15
Cooperative Bank            0.0792926     0.0        0.0792926
First Bank                  0.0157307     0.0        0.0157307
Hua Nan Bank                0.210872      0.207607   0.00326531
Chang Hwa Bank              0.31642       0.31642   -7.18524e-16
Mega Bank                  -5.46585e-15   0.0       -1.57902e-14
Cathay United Bank          0.44

In [53]:
dearevenuegda(X, Y, P, :ERG, names = banks)

General Direct Approach Revenue DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Returns to Scale = VRS
Associated efficiency measure = ERG
─────────────────────────────────────────────────────────────
                              Revenue  Technical   Allocative
─────────────────────────────────────────────────────────────
Export-Import Bank          0.0         0.0       0.0
Bank of Taiwan              0.0         0.0       0.0
Taipei Fubon Bank           0.164297    0.0       0.164297
Bank of Kaohsiung           3.53388     3.53223   0.0016523
Land Bank                   0.0         0.0       0.0
Cooperative Bank            0.0         0.0       0.0
First Bank                  0.0990394   0.0       0.0990394
Hua Nan Bank                0.139932    0.139545  0.000386895
Chang Hwa Bank              0.219293    0.195248  0.0240449
Mega Bank                   0.0         0.0       0.0
Cathay United Bank          1.0596      1.05484   0.00476077
The Shanghai Bank           0.298085    0.124

In [54]:
dearussell(X, Y, orient = :Input, rts = :VRS,  names = banks)

Russell DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
──────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency     effX1     effX2      effX3          slackY1  slackY2
──────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank           1.0       1.0       1.0       1.0           -9.10433e-12      0.0
Bank of Taiwan               1.0       1.0       1.0       1.0            0.0              0.0
Taipei Fubon Bank            1.0       1.0       1.0       1.0            0.0              0.0
Bank of Kaohsiung            0.612019  0.668261  0.563576  0.604219    8535.19             0.0
Land Bank                    1.0       1.0       1.0       1.0            0.0              0.0
Cooperative Bank             1.0       1.0       1.0       1.0            0.0              0.0
First Bank                   1.0       1.0     

In [56]:
dearussell(X, Y, orient = :Output, rts = :VRS,  names = banks)

Russell DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Output; Returns to Scale = VRS
─────────────────────────────────────────────────────────────────────────────────────────────────────
                           efficiency     effY1    effY2          slackX1     slackX2         slackX3
─────────────────────────────────────────────────────────────────────────────────────────────────────
Export-Import Bank            1.0       1.0      1.0          0.0             0.0         0.0
Bank of Taiwan                1.0       1.0      1.0         -1.52296e-10     0.0        -1.45519e-11
Taipei Fubon Bank             1.0       1.0      1.0          0.0             0.0         0.0
Bank of Kaohsiung             4.53223   8.06446  1.0        147.515           0.0        -1.47978e-12
Land Bank                     1.0       1.0      1.0          0.0             0.0         0.0
Cooperative Bank              1.0       1.0      1.0          0.0             0.0         0.0
First Bank     

# 6. Advanced options

## 6.1. Using a different solver

In [ ]:
Pkg.add("Ipopt")
using Ipopt

In [65]:
myoptimizer = DEAOptimizer(Ipopt.Optimizer, time_limit = 10, silent = true);

deacost(X, Y, W, names = banks, optimizer = myoptimizer)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


Cost DEA Model 
DMUs = 31; Inputs = 3; Outputs = 2
Orientation = Input; Returns to Scale = VRS
──────────────────────────────────────────────────────────
                               Cost  Technical  Allocative
──────────────────────────────────────────────────────────
Export-Import Bank         0.999998   0.999999    0.999999
Bank of Taiwan             1.0        1.0         1.0
Taipei Fubon Bank          0.9216     1.0         0.9216
Bank of Kaohsiung          0.597026   0.647333    0.922285
Land Bank                  1.0        1.0         1.0
Cooperative Bank           0.960035   1.0         0.960035
First Bank                 0.989871   1.0         0.989871
Hua Nan Bank               0.81141    0.943612    0.859897
Chang Hwa Bank             0.70393    0.935063    0.752816
Mega Bank                  1.0        1.0         1.0
Cathay United Bank         0.543989   0.709586    0.766629
The Shanghai Bank          0.787542   0.886617    0.888255
Union Bank                 0.31538   

## 6.2. Accessing results

In [66]:
model1 = deaprofitrussell(X, Y, W, P, names = banks);

In [67]:
efficiency(model1, :Economic)

31-element Vector{Float64}:
   3.0391108923815352e-5
   0.17983501594536466
   0.22001922607792812
   4.957206309409705
   1.3702950419996339e-7
   0.28473804077016246
   0.23868854569104117
   0.24015691526021593
   0.44490999895175604
   9.614945777992197e-8
   0.8147914720459225
   1.34536874844074
  25.11526329237874
   ⋮
   6.5393588415663535
   0.49454429279451206
  47.148646760602084
 125.035282407008
   0.6634552135758668
  23.7745207962115
   2.8640898839080084
 136.01042820514445
  43.612876314755574
   1.5644900882674457
   0.4221279505394159
   9.67215044340051

In [68]:
efficiency(model1, :Technical)

31-element Vector{Float64}:
 4.813465512176762e-6
 2.7176397798811536e-9
 2.3459150000348927e-7
 0.3195454766859903
 5.045728190822274e-8
 2.9702607262649394e-8
 1.1901096219624208e-8
 0.12456421692504316
 0.21119900678961423
 2.1158563345835546e-8
 0.33481772743194893
 0.15361118680391705
 0.5399876353001531
 ⋮
 0.32275678010315123
 4.3204104427374546e-7
 0.5355539616272312
 0.6152685737281816
 0.13970392253940211
 0.3619754103054784
 5.084797318755108e-7
 0.5444496334706193
 0.5286020402320507
 1.7079032784916848e-5
 0.12522399717845045
 0.4353758930341637

In [69]:
efficiency(model1, :Allocative)

31-element Vector{Float64}:
   2.557764341163859e-5
   0.17983501322772488
   0.22001899148642812
   4.637660832723715
   8.657222229174065e-8
   0.2847380110675552
   0.23868853378994495
   0.11559269833517277
   0.2337109921621418
   7.499089443408642e-8
   0.4799737446139736
   1.1917575616368228
  24.575275657078585
   ⋮
   6.2166020614632025
   0.4945438607534678
  46.613092798974854
 124.42001383327981
   0.5237512910364647
  23.41254538590602
   2.864089375428277
 135.46597857167384
  43.08427427452352
   1.5644730092346608
   0.2969039533609654
   9.236774550366345

In [70]:
peersmatrix(model1)

31×31 SparseArrays.SparseMatrixCSC{Float64, Int64} with 961 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡇
⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠿⠇